In [1]:
import numpy as np

In [32]:
components = {
    'resources': {'type': 'state', 'value': 1000.0},
    'growth': {'type': 'state', 'value': 0.0},
    'light460nm': {'type': 'stimulus', 'value': 0.0},
    'gene_nMag_T7RNAPn': {'type': 'gene', 'value': True},
    'gene_pMag_T7RNAPc': {'type': 'gene', 'value': True},
    'gene_pT7_CAR': {'type': 'gene', 'value': True},
    'prot_nMag_T7RNAPn': {'type': 'protein', 'value': 0.0},
    'prot_pMag_T7RNAPc': {'type': 'protein', 'value': 0.0},
    'prot_Mag_T7RNAP': {'type': 'protein', 'value': 0.0},
    'prot_CAR': {'type': 'protein', 'value': 0.0},
    'prot_CAR_bound': {'type': 'protein', 'value': 0.0},
}

In [33]:
reactions = {
    'constitutive nMag_T7RNAPn expression': {
        'reactants': {'gene_nMag_T7RNAPn': 0.0, 'resources': -1.0},
        'products': {'prot_nMag_T7RNAPn': 1.0},
        'modifiers': None
    },
    'constitutive pMag_T7RNAPc expression': {
        'reactants': {'gene_pMag_T7RNAPc': 0.0, 'resources': -1.0},
        'products': {'prot_pMag_T7RNAPc': 1.0},
        'modifiers': None
    },
    'light induced nMag-pMag dimerization': {
        'reactants': {'prot_nMag_T7RNAPn': -1.0, 'prot_pMag_T7RNAPc': -1.0},
        'products': {'prot_Mag_T7RNAP': 1.0},
        'modifiers': {'light460nm': 'activator'}
    },
    'T7RNAP driven CAR expression': {
        'reactants': {'resources': -1.0},
        'products': {'prot_CAR': 1.0},
        'modifiers': {'prot_Mag_T7RNAP': 'activator'}
    },
    'T-cell exhaustion': {
        'reactants': {'prot_CAR': -1.0, 'resources': -1.0},
        'products': None,
        'modifiers': {'prot_CAR_bound': 'activator'}
    },
    'T-cell proliferation': {
        'reactants': {'resources': -10.0},
        'products': {'growth': 1.0},
        'modifiers': None
    },
    'metabolism': {
        'reactants': None,
        'products': {'resources': 5.0},
        'modifiers': None
    },
}

In [34]:
class SubCellularNetwork(object):
    """Dynamics of subcellular components - genes, proteins, etc."""
    
    def __init__(self):
        self.components = {}
        self.reactions = {}
        
    def add_components(self, components):
        self.components.update(components)
    
    def remove_components(self, components):
        [self.components.pop(component, None) for component in components]
        
    def add_reactions(self, reactions):
        self.reactions.update(reactions)
    
    def remove_reactions(self, reactions):
        [self.components.pop(reaction, None) for reaction in reactions]

    def react(self, reactants, products, modifiers):
        polarity_mapping = {'activator': 1.0, 'inhibitor': -1.0}
        rate_modifier = 1.0
        if modifiers:
            for modifier, polarity in modifiers.items():
                if type(self.components[modifier]['value']) == bool:
                    rate_modifier = float(self.components[modifier]['value'])
                else:
                    rate_modifier = polarity_mapping[polarity] * np.tanh(self.components[modifier]['value'])
        if reactants:
            for reaction_name, reaction_rate in reactants.items():
                self.components[reaction_name]['value'] += reaction_rate * rate_modifier
        if products:
            for product_name, product_rate in products.items():
                self.components[product_name]['value'] += product_rate * rate_modifier
            
    def update(self):
        [self.react(rxn['reactants'], rxn['products'], rxn['modifiers']) for rxn in self.reactions.values()]

In [35]:
scn = SubCellularNetwork()

In [36]:
scn.add_components(components)

In [37]:
scn.components

{'gene_nMag_T7RNAPn': {'type': 'gene', 'value': True},
 'gene_pMag_T7RNAPc': {'type': 'gene', 'value': True},
 'gene_pT7_CAR': {'type': 'gene', 'value': True},
 'growth': {'type': 'state', 'value': 0.0},
 'light460nm': {'type': 'stimulus', 'value': 0.0},
 'prot_CAR': {'type': 'protein', 'value': 0.0},
 'prot_CAR_bound': {'type': 'protein', 'value': 0.0},
 'prot_Mag_T7RNAP': {'type': 'protein', 'value': 0.0},
 'prot_nMag_T7RNAPn': {'type': 'protein', 'value': 0.0},
 'prot_pMag_T7RNAPc': {'type': 'protein', 'value': 0.0},
 'resources': {'type': 'state', 'value': 1000.0}}

In [38]:
scn.add_reactions(reactions)

In [39]:
scn.reactions

{'T-cell exhaustion': {'modifiers': {'prot_CAR_bound': 'activator'},
  'products': None,
  'reactants': {'prot_CAR': -1.0, 'resources': -1.0}},
 'T-cell proliferation': {'modifiers': None,
  'products': {'growth': 1.0},
  'reactants': {'resources': -10.0}},
 'T7RNAP driven CAR expression': {'modifiers': {'prot_Mag_T7RNAP': 'activator'},
  'products': {'prot_CAR': 1.0},
  'reactants': {'resources': -1.0}},
 'constitutive nMag_T7RNAPn expression': {'modifiers': None,
  'products': {'prot_nMag_T7RNAPn': 1.0},
  'reactants': {'gene_nMag_T7RNAPn': 0.0, 'resources': -1.0}},
 'constitutive pMag_T7RNAPc expression': {'modifiers': None,
  'products': {'prot_pMag_T7RNAPc': 1.0},
  'reactants': {'gene_pMag_T7RNAPc': 0.0, 'resources': -1.0}},
 'light induced nMag-pMag dimerization': {'modifiers': {'light460nm': 'activator'},
  'products': {'prot_Mag_T7RNAP': 1.0},
  'reactants': {'prot_nMag_T7RNAPn': -1.0, 'prot_pMag_T7RNAPc': -1.0}},
 'metabolism': {'modifiers': None,
  'products': {'resources': 

In [43]:
%%timeit
scn.update()

26 µs ± 247 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [44]:
scn.components

{'gene_nMag_T7RNAPn': {'type': 'gene', 'value': 1.0},
 'gene_pMag_T7RNAPc': {'type': 'gene', 'value': 1.0},
 'gene_pT7_CAR': {'type': 'gene', 'value': True},
 'growth': {'type': 'state', 'value': 81113.0},
 'light460nm': {'type': 'stimulus', 'value': 0.0},
 'prot_CAR': {'type': 'protein', 'value': 0.0},
 'prot_CAR_bound': {'type': 'protein', 'value': 0.0},
 'prot_Mag_T7RNAP': {'type': 'protein', 'value': 0.0},
 'prot_nMag_T7RNAPn': {'type': 'protein', 'value': 81113.0},
 'prot_pMag_T7RNAPc': {'type': 'protein', 'value': 81113.0},
 'resources': {'type': 'state', 'value': -566791.0}}

In [ ]:
# Need upper and lower bounds for components